In [ ]:
import sklearn
import numpy as np
import pandas

import sklearn
from sklearn import tree
from sklearn import ensemble

import skimage.data
import skimage.transform
from skimage import data, io, filters

In [ ]:
import math

width = 224
height = 168
# Import Labels
labels = []
imagePaths = []
lastTrackingAngle = 0
lastSpeed = 0
for i in range(20,26):
    fname = "G:/CarDataNew/AccelerationData/capture"+str(i)+"/accelClass.txt"
    base = "G:/CarDataNew/AccelerationData/capture"+str(i) + "/"
    with open(fname) as f:
        content = f.readlines()
        for line in content:
            line = line.split(",")
            speedString = (line[1].split(" "))[2]
            speed = float(speedString)
            trackingAngleString = (line[8].split(" "))[2]
            trackingAngle = float(trackingAngleString)
            deltaTrackingAngle = trackingAngle - lastTrackingAngle
            deltaSpeed = speed - lastSpeed
            lastTrackingAngle = trackingAngle
            lastSpeed = speed
            filePath = (line[10])
            if speed < 60 and abs(deltaTrackingAngle) > 0 and abs(deltaTrackingAngle) < 7:
                if(deltaTrackingAngle>0):
                    deltaTrackingAngle = 1
                else:
                    deltaTrackingAngle = -1
                labels.append(speed)
                #labels.append(deltaTrackingAngle)
                #print speed
                #print trackingAngle
                picturePath = base+filePath
                #print picturePath
                imagePaths.append(picturePath)

trainingIndex = int(len(imagePaths)/1.4)

In [ ]:
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value
from skimage import filters

from skimage.color import rgb2gray
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value
from skimage import filters
import operator
from operator import sub


@adapt_rgb(each_channel)
def sobel_each(image):
    return filters.sobel(image)

def as_gray(image_filter, image, *args, **kwargs):
    gray_image = rgb2gray(image)
    return image_filter(gray_image, *args, **kwargs)

@adapt_rgb(as_gray)
def sobel_gray(image):
    return filters.sobel(image)



images = np.zeros((len(imagePaths),height,width))
print(images.shape)
threadCount = 20
import threading

def worker(loadPath, loadPath2, imageIndex):
    """thread worker function"""
    image = skimage.data.imread(loadPath)
    image2 = skimage.data.imread(loadPath2)
    image = skimage.transform.resize(image, (height, width), mode='reflect')
    image2 = skimage.transform.resize(image2, (height, width), mode='reflect')
    edges = sobel_gray(image)
    edges2 = sobel_gray(image2)
    edges3 = edges2-edges
    images[imageIndex] = edges3
    return edges3, imageIndex

for imNum in range(0,len(imagePaths),threadCount):
    threads = []
    for i in range(threadCount):
        
        if(imNum+i >= len(imagePaths)):
            continue
        if(imNum+1==0):
            loadPath = imagePaths[mNum+i]
            loadPath2 = imagePaths[imNum+i]
        else:
            loadPath = imagePaths[imNum+i-1]
            loadPath2 = imagePaths[imNum+i]
        t = threading.Thread(target=worker, args=(loadPath,loadPath2,imNum+i))
        threads.append(t)
        t.start()

In [ ]:

image = images[23]
print(images[23])
io.imshow(image)
io.show()

In [ ]:

images = np.asarray(images)
images = images.reshape(len(images),1,width*height)

In [ ]:
labels = np.array(labels)
images = np.array(images)
print("labels: ", labels.shape, "\nimages: ", images.shape)

In [ ]:
maxLength = trainingIndex
global currentPos
currentPos = 0
def next(batch_size):
    """ Return a batch of data. When dataset end is reached, start over.
    """
    #if self.batch_id == len(self.data):
    #    self.batch_id = 0
    #batch_data = (self.data[self.batch_id:min(self.batch_id +
    #                                          batch_size, len(self.data))])
    #batch_labels = (self.labels[self.batch_id:min(self.batch_id +
    #                                          batch_size, len(self.data))])
    #batch_seqlen = (self.seqlen[self.batch_id:min(self.batch_id +
    #                                          batch_size, len(self.data))])
    #self.batch_id = min(self.batch_id + batch_size, len(self.data))
    global currentPos
    currentPos = currentPos + batch_size
    maxPos = currentPos + batch_size
    if(maxPos>maxLength):
        currentPos = 0
        maxPos = currentPos + batch_size
    batch_data = (images[currentPos:maxPos])
    batch_labels = (labels[currentPos:maxPos])
    return batch_data, batch_labels



In [ ]:
maxTestingLength = len(labels)
global currentTestingPos
global minIndex 
minIndex = trainingIndex
currentTestingPos = minIndex

def nextTesting(batch_size):
    global currentTestingPos
    global minIndex
    currentTestingPos = currentTestingPos + batch_size
    maxPos = currentTestingPos + batch_size
    if(maxPos>maxTestingLength):
        currentTestingPos = minIndex
        maxPos = currentTestingPos + batch_size
    batch_data = (images[currentTestingPos:maxPos])
    batch_labels = (labels[currentTestingPos:maxPos])
    return batch_data, batch_labels

In [ ]:
x, y = next(128)
print(x.shape)
print(y)

x, y = nextTesting(128)
print(x.shape)
print(y)

cellInitialized = 0

In [ ]:
'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
Long Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

'''
To classify images using a recurrent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
'''

# Parameters
learning_rate = 0.001
training_iters = 160000
batch_size = 128
display_step = 10

# Network Parameters
n_input = width # MNIST data input (img shape: 28*28)
n_steps = height # timesteps
n_hidden = 512 # hidden layer num of features
n_classes = 1 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}


def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    print(x.shape)
    x = tf.unstack(x, n_steps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

def BiRNN(x, weights, biases):

    # Prepare data shape to match `bidirectional_rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

accuracyFloor = tf.constant(36.0, tf.float32)

# Define loss and optimizer
loss = tf.maximum(tf.reduce_mean(tf.square(tf.subtract(pred, y))),accuracyFloor)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Evaluate model
#correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(pred, y))))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
sess=tf.Session()
sess.run(init)



In [ ]:
training_iters = 240000
step = 1
# Keep training until reach max iterations
while step * batch_size < training_iters:
    batch_x, batch_y = next(batch_size)
    # Reshape data to get 28 seq of 28 elements
    batch_x = batch_x.reshape((batch_size, n_steps, n_input))
    # Run optimization op (backprop)
    sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
    if step % display_step == 0:
        # Calculate batch accuracy
        acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
        # Calculate batch loss
        cost = sess.run(loss, feed_dict={x: batch_x, y: batch_y})
        print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
              "{:.6f}".format(cost) + ", Training Accuracy= " + \
              "{:.5f}".format(acc))
    step += 1
print("Optimization Finished!")


In [ ]:
step = 1
totalMAE = 0
correctDirectionCount = 0
totalCount = 0
# Calculate accuracy for 128 mnist test images
while step * batch_size < (len(images)-trainingIndex):
    test_data, test_label = nextTesting(batch_size)
    test_data = test_data.reshape((batch_size, n_steps, n_input))
    predAccuracy = sess.run(accuracy, feed_dict={x: test_data, y: test_label})
    results = sess.run(pred, feed_dict={x: test_data})
    print(results)
    print(test_label)
    for i in range(0,len(results)):
        element = results[i] * test_label[i]
        if(abs(test_label[i])<1):
            continue
        totalCount+=1
        print(element)
        if element > 0:
            correctDirectionCount+=1
    totalMAE += predAccuracy
    print("Testing Accuracy:", predAccuracy)
    step+=1
print("Total MAE: ", (totalMAE/(step-1)))
print("Turn percentage", (totalCount/((step-1) * batch_size)))
print ("Correct Directional Percentage: ",(correctDirectionCount/totalCount))

In [ ]:
#Speed accuracy

step = 1
totalMAE = 0
correctDirectionCount = 0
totalCount = 0
threshold = 12
# Calculate accuracy for 128 mnist test images
while step * batch_size < (len(images)-trainingIndex):
    test_data, test_label = nextTesting(batch_size)
    test_data = test_data.reshape((batch_size, n_steps, n_input))
    predAccuracy = sess.run(accuracy, feed_dict={x: test_data, y: test_label})
    results = sess.run(pred, feed_dict={x: test_data})
    #print(results)
    #print(test_label)
    for i in range(0,len(results)):
        element = abs(results[i] - test_label[i])
        if(abs(test_label[i])>75):
            continue
        totalCount+=1
        print("Actual:", test_label[i],"Guess:",results[i],"Delta",element)
        if element < threshold:
            correctDirectionCount+=1
    totalMAE += predAccuracy
    print("Testing Accuracy:", predAccuracy)
    step+=1
print("Total MAE: ", (totalMAE/(step-1)))
print("Turn percentage", (totalCount/((step-1) * batch_size)))
print ("Correct Speed within threshold percentage ",(correctDirectionCount/totalCount))